In [1]:
from sklearn.linear_model import LinearRegression,LogisticRegression

In [2]:
import numpy as np

lets do with a simple data first 

In [3]:
from sklearn.datasets import load_iris

In [102]:
logreg = LogisticRegression(penalty='l2',verbose=True)

In [5]:
data = load_iris()

In [6]:
data['target']

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

### ahahaha, i think its just binary classification data, but its a multiple classification

for this we have use non linear model, like knn or tree models 

In [7]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_absolute_error,classification_report

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
model = KNeighborsClassifier(n_neighbors=10)

In [10]:
x_train,x_test,y_train,y_test = train_test_split(data['data'],data['target'],random_state=10)

In [11]:
model.fit(x_train,y_train)

KNeighborsClassifier(n_neighbors=10)

In [12]:
y_pred = model.predict(x_test)

In [13]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       0.94      1.00      0.97        15
           2       1.00      0.92      0.96        12

    accuracy                           0.97        38
   macro avg       0.98      0.97      0.97        38
weighted avg       0.98      0.97      0.97        38



In [85]:
data1 = np.array([np.random.normal((1,10)) for _ in range(1000)])

In [89]:
label = list() 
for i in data1 :
    total = np.sum(i)
    if total > 10 : 
        label.append(1)
    else : label.append(0)

In [100]:
label = np.array(label)

In [104]:
x_train,x_test,y_train,y_test = train_test_split(data1,label,random_state=10)

In [105]:
logreg.fit(x_train,y_train)

c:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(verbose=True)

In [107]:
y_pred = logreg.predict(x_test)

In [108]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        56
           1       1.00      1.00      1.00       194

    accuracy                           1.00       250
   macro avg       1.00      1.00      1.00       250
weighted avg       1.00      1.00      1.00       250



In [153]:
class LogisticRegresion_DIY : 
    def __init__ (self,optimizers='Gd',learning_rate=0.001,decay_momentum=0.9,decay_rmsprop=0.999,epsilon=1e-6) : 
        self.optimizers = optimizers 
        self.learning_rate = learning_rate
        self.weight = None 
        self.bias = None 
        self.decay_momentum = decay_momentum
        self.decay_rmsprop = decay_rmsprop
        self.epsilon = epsilon
    
    def build_weight(self, features):
        self.weight = np.random.randn(features, 1).astype(np.float32)
        self.bias = np.random.randn(1).astype(np.float32)
        self.Momentum_w = np.zeros_like(self.weight)
        self.Momentum_b = np.zeros_like(self.bias)
        self.rmsprop_w = np.zeros_like(self.weight)
        self.rmsprop_b = np.zeros_like(self.bias)

    
    def fit (self,X,Y,epochs=100,verbose=0) : 
        features = X.shape[1]
        if self.weight is None and self.bias is None : 
            self.build_weight(features)
        
        total_loss = 0 
        
        for epoch in range(epochs) :
            x = np.dot(X,self.weight) +  self.bias 
            y_pred = 1 / (1 + np.exp(-x))
            loss = np.mean(-(Y * np.log(y_pred + self.epsilon) + (1 - Y) * np.log(1 - y_pred + self.epsilon)))
            total_loss += loss
            if self.optimizers == 'Gd' : 
                gradient_weight = (-1/len(Y)) * np.dot(X.T,(y_pred-Y))
                gradient_bias = (-1/len(Y)) * np.sum(y_pred-Y)
                self.weight -= self.learning_rate * gradient_weight
                self.bias -= self.learning_rate * gradient_bias

            elif self.optimizers == 'Momentum' : 
                gradient_weight = (-1/len(Y)) * np.dot(X.T,(y_pred-Y))
                gradient_bias = (-1/len(Y)) * np.sum(y_pred-Y)
                self.Momentum_w = self.decay_momentum * self.Momentum_w + (1 - self.decay_momentum) * gradient_weight
                self.Momentum_b = self.decay_momentum * self.Momentum_b + (1 - self.decay_momentum) * gradient_bias
                momentum_w = self.Momentum_w / (1 - self.decay_momentum ** (epoch + 1))
                momentum_b = self.Momentum_b / (1 - self.decay_momentum ** (epoch + 1))
                self.weight -= self.learning_rate * momentum_w
                self.bias -= self.learning_rate *momentum_b
            
            elif self.optimizers == 'Rmsprop' : 
                gradient_weight = (-1/len(Y)) * np.dot(X.T,(y_pred-Y))
                gradient_bias = (-1/len(Y)) * np.sum(y_pred-Y)
                self.rmsprop_w = self.decay_rmsprop * self.rmsprop_w +(1 - self.decay_rmsprop) * (np.power(gradient_weight,2))
                self.rmsprop_b = self.decay_rmsprop * self.rmsprop_b + (1 - self.decay_rmsprop) * (np.power(gradient_bias,2))
                rmsprop_w = self.rmsprop_w / (1 - self.decay_rmsprop ** (epoch + 1))
                rmsprop_b = self.rmsprop_b / (1 - self.decay_rmsprop ** (epoch + 1))
                self.weight -= self.learning_rate / (np.sqrt(rmsprop_w + self.epsilon)) * gradient_weight
                self.bias -= self.learning_rate / (np.sqrt(rmsprop_b + self.epsilon)) * gradient_bias 
            
            elif self.optimizers == "Adam" : 
                gradient_weight = (-1/len(Y)) * np.dot(X.T,(y_pred - Y))
                gradient_bias = (-1/len(Y)) * np.sum(y_pred - Y)
                self.Momentum_w = self.decay_momentum * self.Momentum_w + (1 - self.decay_momentum) * gradient_weight
                self.Momentum_b = self.decay_momentum * self.Momentum_b + (1 - self.decay_momentum) * gradient_bias
                self.rmsprop_w = self.decay_rmsprop * self.rmsprop_w + (1 - self.decay_rmsprop) * (np.power(gradient_weight,2))
                self.rmsprop_b = self.decay_rmsprop * self.rmsprop_b + (1 - self.decay_rmsprop) * (np.power(gradient_bias,2))
                momentum_w = self.Momentum_w / (1 - self.decay_momentum ** (epoch + 1)) 
                momentum_b = self.Momentum_b / (1 - self.decay_momentum ** (epoch + 1))
                rmsprop_w = self.rmsprop_w / (1 - self.decay_rmsprop ** (epoch + 1))
                rmsprop_b = self.rmsprop_b / (1 - self.decay_rmsprop ** (epoch + 1))
                self.weight -= self.learning_rate / (np.sqrt(rmsprop_w + self.epsilon)) * momentum_w
                self.bias -= self.learning_rate / (np.sqrt(rmsprop_b + self.epsilon)) * momentum_b
            if verbose != 0 : 
                print(f"epoch : {epoch} || loss : {loss:.6f}")
    
    def predict(self,X_target) : 
        score = np.dot(X_target,self.weight) + self.bias 
        return 1 / (1 + np.exp(-score))


its great for classification but i am still think its almost be overfitting.. but however, i want train regresor 
Models too. so i skip for search better models, cause its just simple dataset, its so easy for machine understanding 
data pattern

In [159]:
models = LogisticRegresion_DIY(optimizers='Adam')

In [127]:
dat = np.dot(x_train.T,np.random.normal(0,1))

In [157]:
models.weight

array([[0.05967926],
       [0.6775275 ]], dtype=float32)

In [164]:
models.fit(x_train,y_train,verbose=1)

epoch : 0 || loss : 1.652183
epoch : 1 || loss : 1.659605
epoch : 2 || loss : 1.665126
epoch : 3 || loss : 1.669863
epoch : 4 || loss : 1.674167
epoch : 5 || loss : 1.678203
epoch : 6 || loss : 1.682062
epoch : 7 || loss : 1.685799
epoch : 8 || loss : 1.689453
epoch : 9 || loss : 1.693051
epoch : 10 || loss : 1.696611
epoch : 11 || loss : 1.700149
epoch : 12 || loss : 1.703676
epoch : 13 || loss : 1.707201
epoch : 14 || loss : 1.710733
epoch : 15 || loss : 1.714276
epoch : 16 || loss : 1.717836
epoch : 17 || loss : 1.721417
epoch : 18 || loss : 1.725023
epoch : 19 || loss : 1.728657
epoch : 20 || loss : 1.732320
epoch : 21 || loss : 1.736016
epoch : 22 || loss : 1.739746
epoch : 23 || loss : 1.743511
epoch : 24 || loss : 1.747313
epoch : 25 || loss : 1.751153
epoch : 26 || loss : 1.755031
epoch : 27 || loss : 1.758950
epoch : 28 || loss : 1.762908
epoch : 29 || loss : 1.766907
epoch : 30 || loss : 1.770948
epoch : 31 || loss : 1.775030
epoch : 32 || loss : 1.779153
epoch : 33 || loss :

In [165]:
models.predict(x_test)

array([[0.05452893],
       [0.18816352],
       [0.20077903],
       [0.05800163],
       [0.09269983],
       [0.11138293],
       [0.07952498],
       [0.01372669],
       [0.03616092],
       [0.02649345],
       [0.22670301],
       [0.25094884],
       [0.28964694],
       [0.05036781],
       [0.10843353],
       [0.01718281],
       [0.01643108],
       [0.01296657],
       [0.03260874],
       [0.08607614],
       [0.02839161],
       [0.23283755],
       [0.0133883 ],
       [0.02409073],
       [0.2189875 ],
       [0.00743991],
       [0.10113475],
       [0.06525636],
       [0.26342645],
       [0.03409139],
       [0.19457666],
       [0.06094131],
       [0.04529698],
       [0.03914651],
       [0.37972572],
       [0.03291702],
       [0.02710638],
       [0.04827505],
       [0.44359269],
       [0.04507273],
       [0.0514107 ],
       [0.03394267],
       [0.07267279],
       [0.02908947],
       [0.0253572 ],
       [0.0738819 ],
       [0.12428503],
       [0.016

In [116]:
x_train[0]

array([ 0.5557324 , 10.34064642])

In [15]:
import pandas as pd

In [16]:
from sklearn.datasets import load_diabetes

In [17]:
data = load_diabetes()

In [18]:
df = pd.DataFrame(data=data['data'],columns=data['feature_names'])

In [19]:
df

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641
...,...,...,...,...,...,...,...,...,...,...
437,0.041708,0.050680,0.019662,0.059744,-0.005697,-0.002566,-0.028674,-0.002592,0.031193,0.007207
438,-0.005515,0.050680,-0.015906,-0.067642,0.049341,0.079165,-0.028674,0.034309,-0.018114,0.044485
439,0.041708,0.050680,-0.015906,0.017293,-0.037344,-0.013840,-0.024993,-0.011080,-0.046883,0.015491
440,-0.045472,-0.044642,0.039062,0.001215,0.016318,0.015283,-0.028674,0.026560,0.044529,-0.025930


In [20]:
x_train,x_test,y_train,y_test = train_test_split(data['data'],data['target'],random_state=100)

In [21]:
y_train = np.array(y_train).reshape(-1,1)

In [22]:
models.fit(x_train,y_train)

LinearRegression(n_jobs=-1)

In [23]:
y_pred = models.predict(x_test)

In [24]:
mean_absolute_error(y_test,y_pred)

40.73965982665262

wow, i make a big mistake in here.. hhmm.. hmm.. I dont care, it just experiment. but i will do my best for solve this problem. firts regresion linear just work buy weight and bias by one line. but x data and target data have so big different range on there, for make it better i'll do preprocessing data for target data

In [25]:
from sklearn.preprocessing import MinMaxScaler

In [26]:
x = data['data']
y = data['target']

In [27]:
minmax = MinMaxScaler(feature_range=(0.1,3))

In [28]:
y = minmax.fit_transform(np.array(y).reshape(-1,1))

In [29]:
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=10)

In [30]:
linear_reg = LinearRegression()

In [31]:
linear_reg.fit(x_train,y_train)

LinearRegression()

In [32]:
y_pred = linear_reg.predict(x_test)

In [33]:
mean_absolute_error(y_test,y_pred)

0.39495656498138454

## ^^ Wow look at that!!  we dit it !! .. we succes make better our models, and just do it by Linear Regression
but actually we can make better again, we can do that by dimention reduction 

In [34]:
from sklearn.decomposition import PCA

In [35]:
pca = PCA(n_components=3)

In [36]:
x = pca.fit_transform(x)

In [37]:
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=10)

In [38]:
linar_model = LinearRegression()

In [39]:
linar_model.fit(x_train,y_train)

LinearRegression()

In [40]:
y_pred = linar_model.predict(x_test)

In [41]:
mean_absolute_error(y_test,y_pred)

0.4735975731511531

:< huhuhu.. i make mistake again, hhmmm.. yeah.. you know, its failure cause.. my stupid solve way.. yeah, if your all have another way you can do it by your self
yeah.. okay okay.. i will do by another models, we will use KNN Models or tree Models, but we use KNN first

In [42]:
from sklearn.neighbors import KNeighborsRegressor

In [43]:
data = load_diabetes()

In [44]:
models = KNeighborsRegressor()

In [45]:
x = data['data']
y = minmax.transform(np.array(data['target'].reshape(-1,1)))

In [46]:
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=10)

In [47]:
models.fit(x_train,y_train)

KNeighborsRegressor()

In [48]:
y_pred = models.predict(x_test)

In [49]:
mean_absolute_error(y_test,y_pred)

0.4397165389688754

# okey, next use Tree Models...

In [50]:
from sklearn.tree import DecisionTreeRegressor

In [51]:
tree = DecisionTreeRegressor(criterion='absolute_error')

In [52]:
tree.fit(x_train,y_train)

DecisionTreeRegressor(criterion='absolute_error')

In [53]:
y_pred = tree.predict(x_test)

In [54]:
mean_absolute_error(y_test,y_pred)

0.5641127108416829

arghh.. i failure, i give up.. 

In [55]:
class Reg_Linear : 
    def __init__ (self,optimizers='Gd',lr = 0.001,decay_moment = 0.9,decay_rmsprop = 0.999,epsilon=1e-7) : 
        self.optimizer = optimizers
        self.weight = None 
        self.Bias = None 
        self.learning_rate = lr 
        self.decay_momentum = decay_moment
        self.decay_rmsprop = decay_rmsprop
        self.epsilon = epsilon

    def weight_build (self,features) : 
        self.weight = np.random.rand(features,1)
        self.Bias = np.random.rand(1)
        self.Momentum_w = np.zeros_like(self.weight)
        self.Momentum_b = np.zeros_like(self.Bias)
        self.rmsprop_w = np.zeros_like(self.weight)
        self.rmsprop_b = np.zeros_like(self.Bias)

    def fit(self,X,Y,loss='mse',epochs=50) : 
        features = X.shape[1]
        if self.weight is None and self.Bias is None : 
            self.weight_build(features)
        total_loss = 0 
        for epoch in range(epochs) :
            losses = 0
            y_pred = np.dot(X, self.weight) + self.Bias
            if loss == 'mse' : 
                losses = np.mean(np.power((Y-y_pred),2))
                total_loss += losses
            elif loss == 'mae' : 
                losses = np.mean(np.abs(Y-y_pred))
                total_loss += losses
            if self.optimizer == 'Adam' : 
                self.gradient_weight = (-2/len(Y)) * np.dot(X.T,(Y - y_pred))
                self.gradient_bias = (-2/len(Y)) * np.sum(Y - y_pred)
                self.Momentum_w = self.decay_momentum * self.Momentum_w + (1 - self.decay_momentum) * self.gradient_weight
                self.Momentum_b = self.decay_momentum * self.Momentum_b + (1 - self.decay_momentum) * self.gradient_bias
                self.rmsprop_w = self.decay_rmsprop * self.rmsprop_w + (1 - self.decay_rmsprop) * (np.power(self.gradient_weight,2))
                self.rmsprop_b = self.decay_rmsprop * self.rmsprop_b + (1 - self.decay_rmsprop) * (np.power(self.gradient_bias,2))
                moment_w_correct = self.Momentum_w / (1 - self.decay_momentum**(epoch + 1))
                rms_w_correct = self.rmsprop_w / (1 - self.decay_rmsprop ** (epoch + 1))
                moment_b_correct = self.Momentum_b / (1 - self.decay_momentum ** (epoch + 1))
                rms_b_correct = self.rmsprop_b / (1 - self.decay_rmsprop ** (epoch + 1))
                self.weight -= (self.learning_rate / np.sqrt(rms_w_correct + self.epsilon)) * moment_w_correct
                self.Bias -= (self.learning_rate / np.sqrt(rms_b_correct + self.epsilon)) * moment_b_correct
            elif self.optimizer == 'Gd' : 
                self.gradient_weight = (-2/len(Y)) * np.dot( X.T, (Y -y_pred))
                self.gradient_bias = (-2/len(Y)) * np.sum(Y - y_pred)
                self.weight -= self.learning_rate * (self.gradient_weight)
                self.Bias -= self.learning_rate * (self.gradient_bias)
            print(f"epochs : {epoch} || loss : {total_loss:.6f}")
    
    def predict(self,X_Target) : 
        return np.dot(X_Target,self.weight) + self.Bias



In [56]:
data = load_diabetes()

In [57]:
Train_data = data.copy()

In [58]:
x = Train_data['data']
y = Train_data['target']

In [59]:
np.dot(np.random.rand(10),x[0]) + np.random.rand()

0.6825156983653369

In [60]:
len(x)

442

In [61]:
y = minmax.transform(np.array(y).reshape(-1,1))

In [62]:
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=10)

In [63]:
Reg_linear_DIY = Reg_Linear(optimizers='Adam',lr=0.001,epsilon=1e-8)

In [64]:
Reg_linear_DIY.fit(x_train,y_train,epochs=10000)

epochs : 0 || loss : 1.114677
epochs : 1 || loss : 2.227502
epochs : 2 || loss : 3.338477
epochs : 3 || loss : 4.447603
epochs : 4 || loss : 5.554884
epochs : 5 || loss : 6.660321
epochs : 6 || loss : 7.763918
epochs : 7 || loss : 8.865676
epochs : 8 || loss : 9.965599
epochs : 9 || loss : 11.063688
epochs : 10 || loss : 12.159946
epochs : 11 || loss : 13.254377
epochs : 12 || loss : 14.346982
epochs : 13 || loss : 15.437764
epochs : 14 || loss : 16.526726
epochs : 15 || loss : 17.613870
epochs : 16 || loss : 18.699200
epochs : 17 || loss : 19.782719
epochs : 18 || loss : 20.864428
epochs : 19 || loss : 21.944331
epochs : 20 || loss : 23.022431
epochs : 21 || loss : 24.098731
epochs : 22 || loss : 25.173233
epochs : 23 || loss : 26.245941
epochs : 24 || loss : 27.316858
epochs : 25 || loss : 28.385985
epochs : 26 || loss : 29.453328
epochs : 27 || loss : 30.518887
epochs : 28 || loss : 31.582667
epochs : 29 || loss : 32.644671
epochs : 30 || loss : 33.704901
epochs : 31 || loss : 34.76

In [65]:
Reg_linear_DIY.gradient_bias

-1.0318676893214789e-06

In [66]:
Reg_linear_DIY.Momentum_b

array([-1.03645648e-06])

In [67]:
y_pred = Reg_linear_DIY.predict(x_test)

In [68]:
y_pred[3]

array([0.66126513])

In [69]:
mean_absolute_error(y_test,y_pred)

0.3943904449458071

In [70]:
y_pred2 = Reg_linear_DIY.predict(x)

In [71]:
y_pred_valid = minmax.inverse_transform(y_pred2)

In [72]:
mean_absolute_error(data['target'],y_pred_valid)

43.41808057391626

In [73]:
data['target']

array([151.,  75., 141., 206., 135.,  97., 138.,  63., 110., 310., 101.,
        69., 179., 185., 118., 171., 166., 144.,  97., 168.,  68.,  49.,
        68., 245., 184., 202., 137.,  85., 131., 283., 129.,  59., 341.,
        87.,  65., 102., 265., 276., 252.,  90., 100.,  55.,  61.,  92.,
       259.,  53., 190., 142.,  75., 142., 155., 225.,  59., 104., 182.,
       128.,  52.,  37., 170., 170.,  61., 144.,  52., 128.,  71., 163.,
       150.,  97., 160., 178.,  48., 270., 202., 111.,  85.,  42., 170.,
       200., 252., 113., 143.,  51.,  52., 210.,  65., 141.,  55., 134.,
        42., 111.,  98., 164.,  48.,  96.,  90., 162., 150., 279.,  92.,
        83., 128., 102., 302., 198.,  95.,  53., 134., 144., 232.,  81.,
       104.,  59., 246., 297., 258., 229., 275., 281., 179., 200., 200.,
       173., 180.,  84., 121., 161.,  99., 109., 115., 268., 274., 158.,
       107.,  83., 103., 272.,  85., 280., 336., 281., 118., 317., 235.,
        60., 174., 259., 178., 128.,  96., 126., 28

In [74]:
y

array([[1.23831776],
       [0.5517134 ],
       [1.14797508],
       [1.73520249],
       [1.09376947],
       [0.75046729],
       [1.12087227],
       [0.44330218],
       [0.86791277],
       [2.67476636],
       [0.78660436],
       [0.49750779],
       [1.49127726],
       [1.54548287],
       [0.94018692],
       [1.41900312],
       [1.37383178],
       [1.17507788],
       [0.75046729],
       [1.39190031],
       [0.48847352],
       [0.31682243],
       [0.48847352],
       [2.08753894],
       [1.5364486 ],
       [1.69906542],
       [1.11183801],
       [0.64205607],
       [1.0576324 ],
       [2.43084112],
       [1.03956386],
       [0.40716511],
       [2.95482866],
       [0.66012461],
       [0.46137072],
       [0.79563863],
       [2.2682243 ],
       [2.36760125],
       [2.15077882],
       [0.68722741],
       [0.77757009],
       [0.37102804],
       [0.42523364],
       [0.70529595],
       [2.21401869],
       [0.3529595 ],
       [1.59065421],
       [1.157